In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abhishektiwari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import re
import pandas as pd
import numpy as np
from nltk.tokenize import  word_tokenize 

In [5]:
datatoclean = pd.read_csv(f'tf_idf_dataset.csv')


In [6]:
#datatoclean.info()
#datatoclean
def clean_data(text):
  #text = text.lower()
  text = re.sub("_"," ",text)
  text = re.sub("@[A-Za-z0-9_]+","", text) #mentions
  text = re.sub("#[A-Za-z0-9_]+","", text) #hashtags
  text = re.sub(r"https?:\/\/\S+", "", text) #links
  text = re.sub(r"www.\S+", "", text) #links
  text = re.sub(r"RT[\s]+","",text) #Retweets
  text = re.sub('[()!?]', ' ', text) #punctuations
  text = re.sub('\[.*?\]',' ', text)
  text = re.sub("[0-9_]+","",text) #numbers
  text = re.sub("\n","",text) #spacing
  text = re.sub(r'\b\w{1,3}\b', '', text);
  text = re.sub(r'^\s+", "', "a\n b\n c", text)
  punctuations = '@#!?+&*,[]-%.:/();$=><|{}^»«""'
  for p in punctuations:
      text = text.replace(p, '')
  text = text.replace("\\"," ")
  text = text.replace("'"," ") #apostrophe
  text = text.replace("’"," ") #apostrophe
  return text

In [7]:
datatoclean["product_name"] = datatoclean["product_name"].apply(lambda x: clean_data(x.strip()))

In [8]:
#Example text corpus for our tutorial
text = datatoclean["product_name"]
 
#Preprocessing the text data
sentences = []
word_set = []
 
for sent in text:
    x = [i.lower() for  i in word_tokenize(sent) if i.isalpha()]
    sentences.append(x)
    for word in x:
        if word not in word_set:
            word_set.append(word)
 
#Set of vocab 
word_set = set(word_set)
#Total documents in our corpus
total_documents = len(sentences)
 
#Creating an index for each word in our vocab.
index_dict = {} #Dictionary to store index for each word
i = 0
for word in word_set:
    index_dict[word] = i
    i += 1

In [9]:
#Create a count dictionary
 
def count_dict(sentences):
    word_count = {}
    for word in word_set:
        word_count[word] = 0
        for sent in sentences:
            if word in sent:
                word_count[word] += 1
    return word_count
 
word_count = count_dict(sentences)

In [10]:
#Term Frequency
def termfreq(document, word):
    N = len(document)
    occurance = len([token for token in document if token == word])
    return occurance/N

In [11]:
#Inverse Document Frequency
 
def inverse_doc_freq(word):
    try:
        word_occurance = word_count[word]
    except:
        word_occurance = 1
    return np.log(total_documents/word_occurance)

In [12]:
def tf_idf(sentence):
    tf_idf_vec = np.zeros((len(word_set),))
    for word in sentence:
        tf = termfreq(sentence,word)
        idf = inverse_doc_freq(word)
         
        value = tf*idf
        tf_idf_vec[index_dict[word]] = value
    return tf_idf_vec

In [13]:
vectors = []
for sent in sentences:
    vec = tf_idf(sent)
    vectors.append(vec)
 
#print(vectors)

In [14]:
vector_scratch = np.array(vectors)

In [15]:
#to compare with the scikit learn vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(use_idf=True)
X = vectorizer.fit_transform(text)

In [16]:
X.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.331396  , 0.        ,
         0.        ],
        [0.        , 0.        , 0.26699553, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.48866328]])

In [17]:
vector_scratch = np.array(vectors)
vector_scratch

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
#adding headers
result = pd.DataFrame(vector_scratch)
result.columns = word_set

In [50]:
np.savetxt("vectornltk.csv",vector_scratch,delimiter=",")

In [20]:
np.savetxt("vector.csv",result,delimiter=",") 